In [1]:
# ✓ Классификатор градиентного бустинга. 
# _✓ Классификатор CatBoost. 
# _✓ Классификатор Ada Boost. 
# _✓ Классификатор Extra Trees. 
# ✓ Квадратичный дискриминантный анализ. 	
# ✓ Light Gradient Boosting Machine. 
# -✓ Классификатор K Neighbors.  
# ✓ Классификатор дерева решений. 
# ✓ Экстремальный градиентный бустинг.
# ✓ Фиктивный классификатор.  
# ✓ SVM - линейное ядро.

In [2]:
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,  classification_report

In [3]:
# перезаписать файл .json в .csv
import pandas as pd

df = pd.read_json('train.json')
df.to_csv('train.csv', index=False)

df = pd.read_json('test.json')
df.to_csv('test.csv', index=False)

In [4]:
# добавить столбец 'cuisine' в test 
import pandas as pd

# Загрузка CSV
df = pd.read_csv('test.csv')

# Добавление пустого столбца на позицию `index=1` (второй столбец)
df.insert(1, 'cuisine', None)  # или pd.NA, "", np.nan

# Сохранение
df.to_csv('test.csv', index=False)

In [5]:
# объединяем два csv в один
import pandas as pd

# Загружаем оба CSV
df1 = pd.read_csv('train.csv')
df2 = pd.read_csv('test.csv')

# Объединяем их по строкам
df_combined = pd.concat([df1, df2], ignore_index=True)

# Сохраняем результат в новый CSV
df_combined.to_csv('combined-max.csv', index=False)

# Выводим первые 5 строк для проверки
print(df_combined.head())


      id      cuisine                                        ingredients
0  10259        greek  ['romaine lettuce', 'black olives', 'grape tom...
1  25693  southern_us  ['plain flour', 'ground pepper', 'salt', 'toma...
2  20130     filipino  ['eggs', 'pepper', 'salt', 'mayonaise', 'cooki...
3  22213       indian        ['water', 'vegetable oil', 'wheat', 'salt']
4  13162       indian  ['black pepper', 'shallots', 'cornflour', 'cay...


In [6]:
# 2. Классификатор CatBoost.
!pip install catboost

You should consider upgrading via the '/home/dzmitry/.pyenv/versions/3.10.0/bin/python3.10 -m pip install --upgrade pip' command.


In [7]:
# 2. Классификатор CatBoost на данных, синтетических:
import numpy as np
from catboost import CatBoostClassifier
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

# Загружаем данные из CSV
df = pd.read_csv('combined.csv')

# Проверим названия столбцов
print(df.columns)

# Преобразуем строки в списки ингредиентов
df['ingredients'] = df['ingredients'].apply(lambda x: x.split(', ') if isinstance(x, str) else [])

# Применяем MultiLabelBinarizer для one-hot encoding
mlb = MultiLabelBinarizer()
ingredients_onehot = mlb.fit_transform(df['ingredients'])

# Преобразуем в DataFrame
ingredients_df = pd.DataFrame(ingredients_onehot, columns=mlb.classes_)

# Убираем лишние пробелы в столбцах
df.columns = df.columns.str.strip()

# Объединяем one-hot признаки с оригинальными данными, удаляя столбец ingredients
df_onehot = pd.concat([ingredients_df, df.drop(['ingredients'], axis=1)], axis=1)

# Признаки (X) — это все столбцы, кроме 'cuisine'
X = df_onehot.drop('cuisine', axis=1)

# Целевая переменная (y) — это столбец 'cuisine'
y = df_onehot['cuisine']

# Преобразуем целевую переменную (cuisine) в числовой формат
y = pd.factorize(y)[0]  # Преобразует категориальные значения в числа (например, 'french' -> 0)

# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Создание и обучение классификатора CatBoost
clf = CatBoostClassifier(iterations=100, learning_rate=0.1, depth=3, random_state=42, verbose=0)
clf.fit(X_train, y_train)

# Прогнозируем на тестовых данных
y_pred = clf.predict(X_test)  # <- Исправлено: clf вместо model

# Оцениваем точность модели
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

print(classification_report(y_test, y_pred))

Index(['id', 'cuisine', 'ingredients'], dtype='object')
Accuracy: 0.2880
              precision    recall  f1-score   support

          -1       0.13      0.52      0.21        42
           0       0.00      0.00      0.00         7
           1       0.50      0.07      0.12        14
           2       0.00      0.00      0.00         7
           3       1.00      0.05      0.10        20
           4       0.00      0.00      0.00         4
           5       0.00      0.00      0.00         5
           6       0.50      0.48      0.49        46
           7       0.67      0.59      0.62        34
           8       0.67      0.50      0.57        12
           9       0.00      0.00      0.00         2
          10       0.00      0.00      0.00         3
          11       0.00      0.00      0.00         3
          12       0.00      0.00      0.00        13
          13       0.00      0.00      0.00         3
          14       0.00      0.00      0.00         9
        

/home/dzmitry/.pyenv/versions/3.10.0/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/dzmitry/.pyenv/versions/3.10.0/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/dzmitry/.pyenv/versions/3.10.0/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.cap

In [8]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score



# Загружаем данные из CSV
df = pd.read_csv('combined.csv')

# Проверим названия столбцов
print(df.columns)

# Преобразуем строки в списки ингредиентов
df['ingredients'] = df['ingredients'].apply(lambda x: x.split(', ') if isinstance(x, str) else [])

# Применяем MultiLabelBinarizer для one-hot encoding
mlb = MultiLabelBinarizer()
ingredients_onehot = mlb.fit_transform(df['ingredients'])

# Преобразуем в DataFrame
ingredients_df = pd.DataFrame(ingredients_onehot, columns=mlb.classes_)

# Убираем лишние пробелы в столбцах
df.columns = df.columns.str.strip()

# Объединяем one-hot признаки с оригинальными данными, удаляя столбец ingredients
df_onehot = pd.concat([ingredients_df, df.drop(['ingredients'], axis=1)], axis=1)

# Признаки (X) — это все столбцы, кроме 'cuisine'
X = df_onehot.drop('cuisine', axis=1)

# Целевая переменная (y) — это столбец 'cuisine'
y = df_onehot['cuisine']

# Преобразуем целевую переменную (cuisine) в числовой формат
y = pd.factorize(y)[0]  # Преобразует категориальные значения в числа (например, 'french' -> 0)

# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Создание и обучение классификатора Extra Trees
clf = ExtraTreesClassifier(n_estimators=100, max_features='sqrt', random_state=42)
clf.fit(X_train, y_train)

# Прогнозирование и оценка точности
y_pred = clf.predict(X_test)

# Вывод метрик классификации
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy:.2f}')
print('Classification Report:')
print(report)


Index(['id', 'cuisine', 'ingredients'], dtype='object')
Accuracy: 0.39
Classification Report:
              precision    recall  f1-score   support

          -1       0.17      0.38      0.23        42
           0       0.50      0.14      0.22         7
           1       0.43      0.43      0.43        14
           2       0.00      0.00      0.00         7
           3       0.64      0.35      0.45        20
           4       0.00      0.00      0.00         4
           5       0.00      0.00      0.00         5
           6       0.48      0.59      0.53        46
           7       0.60      0.74      0.66        34
           8       0.59      0.83      0.69        12
           9       0.00      0.00      0.00         2
          10       0.67      0.67      0.67         3
          11       0.00      0.00      0.00         3
          12       1.00      0.15      0.27        13
          13       0.00      0.00      0.00         3
          14       0.33      0.11      0.

/home/dzmitry/.pyenv/versions/3.10.0/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/dzmitry/.pyenv/versions/3.10.0/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/dzmitry/.pyenv/versions/3.10.0/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.cap

In [9]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

# Загружаем данные из CSV
df = pd.read_csv('combined.csv')

# Проверим названия столбцов
print(df.columns)

# Преобразуем строки в списки ингредиентов
df['ingredients'] = df['ingredients'].apply(lambda x: x.split(', ') if isinstance(x, str) else [])

# Применяем MultiLabelBinarizer для one-hot encoding
mlb = MultiLabelBinarizer()
ingredients_onehot = mlb.fit_transform(df['ingredients'])

# Преобразуем в DataFrame
ingredients_df = pd.DataFrame(ingredients_onehot, columns=mlb.classes_)

# Убираем лишние пробелы в столбцах
df.columns = df.columns.str.strip()

# Объединяем one-hot признаки с оригинальными данными, удаляя столбец ingredients
df_onehot = pd.concat([ingredients_df, df.drop(['ingredients'], axis=1)], axis=1)

# Признаки (X) — это все столбцы, кроме 'cuisine'
X = df_onehot.drop('cuisine', axis=1)

# Целевая переменная (y) — это столбец 'cuisine'
y = df_onehot['cuisine']

# Преобразуем целевую переменную (cuisine) в числовой формат
y = pd.factorize(y)[0]  # Преобразует категориальные значения в числа (например, 'french' -> 0)

# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Определение базовой модели (слабой)
base_model = DecisionTreeClassifier(max_depth=1)  # Дерево решений с 1 разделением

# Инициализация AdaBoost
adaboost = AdaBoostClassifier(
    estimator=base_model,
    n_estimators=50,  # Количество слабых моделей
    learning_rate=1.0,  # Скорость обучения
    random_state=42
)

# Обучение модели
adaboost.fit(X_train, y_train)

# Предсказание на тестовых данных
y_pred = adaboost.predict(X_test)


# # Прогнозируем на тестовых данных
# y_pred = model.predict(X_test)

# Оцениваем точность модели
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

print(classification_report(y_test, y_pred))

Index(['id', 'cuisine', 'ingredients'], dtype='object')
Accuracy: 0.2200
              precision    recall  f1-score   support

          -1       0.18      0.95      0.30        42
           0       0.00      0.00      0.00         7
           1       0.00      0.00      0.00        14
           2       0.00      0.00      0.00         7
           3       0.00      0.00      0.00        20
           4       0.00      0.00      0.00         4
           5       0.00      0.00      0.00         5
           6       0.00      0.00      0.00        46
           7       0.58      0.32      0.42        34
           8       1.00      0.33      0.50        12
           9       0.00      0.00      0.00         2
          10       0.00      0.00      0.00         3
          11       0.00      0.00      0.00         3
          12       0.00      0.00      0.00        13
          13       0.00      0.00      0.00         3
          14       0.00      0.00      0.00         9
        

/home/dzmitry/.pyenv/versions/3.10.0/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/dzmitry/.pyenv/versions/3.10.0/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/dzmitry/.pyenv/versions/3.10.0/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.cap

In [11]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neighbors import KNeighborsClassifier

# Загружаем данные из CSV
df = pd.read_csv('combined.csv')

# Проверим названия столбцов
print(df.columns)

# Преобразуем строки в списки ингредиентов
df['ingredients'] = df['ingredients'].apply(lambda x: x.split(', ') if isinstance(x, str) else [])

# Применяем MultiLabelBinarizer для one-hot encoding
mlb = MultiLabelBinarizer()
ingredients_onehot = mlb.fit_transform(df['ingredients'])

# Преобразуем в DataFrame
ingredients_df = pd.DataFrame(ingredients_onehot, columns=mlb.classes_)

# Убираем лишние пробелы в столбцах
df.columns = df.columns.str.strip()

# Объединяем one-hot признаки с оригинальными данными, удаляя столбец ingredients
df_onehot = pd.concat([ingredients_df, df.drop(['ingredients'], axis=1)], axis=1)

# Признаки (X) — это все столбцы, кроме 'cuisine'
X = df_onehot.drop('cuisine', axis=1)

# Целевая переменная (y) — это столбец 'cuisine'
y = df_onehot['cuisine']

# Преобразуем целевую переменную (cuisine) в числовой формат
y = pd.factorize(y)[0]  # Преобразует категориальные значения в числа (например, 'french' -> 0)

# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# from sklearn.neighbors import KNeighborsClassifier

# Создание модели (k=3 по умолчанию)
knn = KNeighborsClassifier(n_neighbors=3)

# Обучение модели
knn.fit(X_train, y_train)

# Предсказание на тестовых данных
y_pred = knn.predict(X_test)

# Оцениваем точность модели
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

print(classification_report(y_test, y_pred))



Index(['id', 'cuisine', 'ingredients'], dtype='object')
Accuracy: 0.1240
              precision    recall  f1-score   support

          -1       0.15      0.40      0.22        42
           0       0.00      0.00      0.00         7
           1       0.09      0.14      0.11        14
           2       0.00      0.00      0.00         7
           3       0.08      0.05      0.06        20
           4       0.00      0.00      0.00         4
           5       0.00      0.00      0.00         5
           6       0.25      0.20      0.22        46
           7       0.13      0.06      0.08        34
           8       0.00      0.00      0.00        12
           9       0.00      0.00      0.00         2
          10       0.00      0.00      0.00         3
          11       0.00      0.00      0.00         3
          12       0.00      0.00      0.00        13
          13       0.00      0.00      0.00         3
          14       0.00      0.00      0.00         9
        

/home/dzmitry/.pyenv/versions/3.10.0/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/dzmitry/.pyenv/versions/3.10.0/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/dzmitry/.pyenv/versions/3.10.0/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.cap